In [1]:
import requests
import json
from pprint import pprint
import pandas as pd

def priceToken0(swap):
    """
    Converting from Uniswap's SqrtPriceX96 quantity
    to a de-sqrt'ed, de-X96'ed, and de-decimalized number.
    
    This gives the price of token1 in units of token0.
    
    Reference: https://docs.uniswap.org/sdk/v3/guides/fetching-prices
    """
    d0 = int(swap["pool"]["token0"]["decimals"])
    d1 = int(swap["pool"]["token1"]["decimals"])

    p_smol = float(swap["sqrtPriceX96"])**2 / (2**192)
    
    p = p_smol * (10**(-d1)) / (10**(-d0))
    
    return p

def priceToken1(swap):
    """
    This gives the price of token1 in units of token0.
    """
    return 1/priceToken0(swap)

def prepswapdata(swaps):    
    rows = []
    for swap in swaps:
        rows.append(
            {
                "amount0": float(swap["amount0"]),
                "amount1": float(swap["amount1"]),
                "boughtToken0": 1 if (float(swap["amount0"]) < 0) else 0, # it's a buy if amount0 is positive
                "price0After": priceToken0(swap),
                "price1After": priceToken1(swap),
                "blockNumber": int(swap["transaction"]["blockNumber"]),
                "blockPosition": -1,
                "blockTimestamp": float(swap["timestamp"]),
                "txnHash": swap["transaction"]["id"],
                "logIndex": int(swap["logIndex"]),
            }
        )
    
    df = pd.DataFrame(rows)
    df = df.sort_values(by=["blockNumber", "logIndex"], ascending=True) # sorts the transactions by occurrence within the block
#     df = df.drop(columns=["logIndex"])
    return df

def getswaps(pool, cutoff_timestamp=1668227632):
    """
    Get all of the Uniswap swaps on a particular pool, back
    to a specific time `cutoff_timestamp`.
    
    pool: str
    cutoff_timestamp: int
    """
    pool = pool.lower().strip()
    url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
    allswaps = []
    sup_timestamp = 10**15 # greates that the timestamp can be, on each round of pagination
    
    for i in range(1_001):
        print("i:", i)
        
        query = '''
        {
            swaps(
                first: 1000, 
                #skip: %d, 
                orderBy: timestamp, 
                orderDirection: desc,
                where: {
                    pool: "%s",
                    timestamp_lte: %d
                    #timestamp_gt: %d
                }
            ) {
                amount0
                amount1
                amountUSD
                timestamp
                sqrtPriceX96
                tick
                logIndex

                pool {
                    token0 {
                        id
                        symbol
                        decimals
                    }
                    token1 {
                        id
                        symbol
                        decimals
                    }
                    #feesUSD
                    #txCount
                    #observationIndex
                    #collectedFeesUSD
                    #collectedFeesToken0
                    #collectedFeesToken1
                }
                
                transaction {
                    blockNumber,
                    id
                }
            }
        }
        ''' % (
            i*1_000,
            pool,
            sup_timestamp,
            cutoff_timestamp
        )

        result = json.loads(requests.post(
            url,
            json={"query": query}
        ).content)

        if ("data" not in result) or (len(result["data"]["swaps"]) == 0):
            pprint(result)
            print("breaking a")
            break

        swaps = result["data"]["swaps"]


        # linear-time search for the last timestamp; can be optimized to O(log(n)), but not worth it rn
        j = len(swaps)
        while int(swaps[j-1]["timestamp"]) < cutoff_timestamp:
            j -= 1

        if sup_timestamp == int(swaps[j-1]["timestamp"]):
            # note, this could be problematic if we had >1,000 swaps that all had the same timestamp
            print("done, since we have the same sup_timestamp as before")
            break
        else:
            sup_timestamp = int(swaps[j-1]["timestamp"]) # set the maximum time
            print("sup timestamp", sup_timestamp, "\n")

        # concatenate all swaps that have timestamp >= cutoff_timestamp
        allswaps += swaps[:j]
        if j != len(swaps):
            print("done, since the final swap is in the interior of the result batch")
            break
            
    pprint(allswaps[-1])
            
    return prepswapdata(allswaps)


In [2]:
swaps_ = getswaps("0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", 1670852039)
swaps_

i: 0
sup timestamp 1671899195 

i: 1
sup timestamp 1671874631 

i: 2
sup timestamp 1671851495 

i: 3
sup timestamp 1671832391 

i: 4
sup timestamp 1671816731 

i: 5
sup timestamp 1671805655 

i: 6
sup timestamp 1671792851 

i: 7
sup timestamp 1671774203 

i: 8
sup timestamp 1671757235 

i: 9
sup timestamp 1671742871 

i: 10
sup timestamp 1671732395 

i: 11
sup timestamp 1671720983 

i: 12
sup timestamp 1671705779 

i: 13
sup timestamp 1671686819 

i: 14
sup timestamp 1671668843 

i: 15
sup timestamp 1671654959 

i: 16
sup timestamp 1671641315 

i: 17
sup timestamp 1671629255 

i: 18
sup timestamp 1671611699 

i: 19
sup timestamp 1671594851 

i: 20
sup timestamp 1671580175 

i: 21
sup timestamp 1671567779 

i: 22
sup timestamp 1671555743 

i: 23
sup timestamp 1671547187 

i: 24
sup timestamp 1671532427 

i: 25
sup timestamp 1671518099 

i: 26
sup timestamp 1671505343 

i: 27
sup timestamp 1671494363 

i: 28
sup timestamp 1671485219 

i: 29
sup timestamp 1671475151 

i: 30
sup timestamp 

,amount0,amount1,boughtToken0,price0After,price1After,blockNumber,blockPosition,blockTimestamp,txnHash,logIndex
83943,346.416583,-0.276745,0,0.000799,1251.128025,16168952,-1,1.670852e+09,0x61f10c815d778ff4587d28b824e5db098f5dc18a9c7b...,297
83942,-996.210738,0.796649,1,0.000799,1251.125042,16168954,-1,1.670852e+09,0xd31cc75a989f6790fcb180e98920002eb8c1d2d32f50...,107
83941,-200.079868,0.160000,1,0.000799,1251.124443,16168955,-1,1.670852e+09,0x549c575c572f90d0c5594df7e15270c6910f27cf2834...,174
83940,54578.856397,-43.599185,0,0.000799,1251.287798,16168957,-1,1.670852e+09,0x6707334d3c3d6bcd7e2ace30db8a811949d407be9705...,71
83938,-187.599281,0.150000,1,0.000799,1251.287236,16168958,-1,1.670852e+09,0xa5792a776f16086de3242e50b68653005cd75e5082af...,52
...,...,...,...,...,...,...,...,...,...,...
4,300.000000,-0.245936,0,0.000820,1219.217751,16257444,-1,1.671919e+09,0x7d07412e72065a15973eff7f7cc31baeb5f5bf10e785...,180
3,-116484.237518,95.600000,1,0.000820,1218.910067,16257445,-1,1.671919e+09,0xbe2761e5ccf7ed7519128257c6023372121d64529216...,66
1,-1013.048760,0.831527,1,0.000820,1218.907391,16257446,-1,1.671919e+09,0xed26e023aee032a8f426f7911b8f5c20e24a36d5d2d0...,30
2,36602.717777,-30.012917,0,0.000820,1219.004022,16257446,-1,1.671919e+09,0x58ef671f7746f1f59ba4323962d72c3bb1e0acbcf181...,95


In [292]:
swaps_[swaps_.blockNumber == 16168749] # 92 then 112, in order

d_ = swaps_[swaps_.blockNumber == 16168966]
for i_ in range(len(d_)):
    print(d_.iloc[i_].txnHash) # txnPosition: [126, 111, 169]; li: [154, 125, 206]
d_

0xb58aee09dd66874b29889f1fb4553279e022e53fd70b04c1e7fb51db22f8f31d
0xf7650797b933a90858f2201422cf5d3be3d90ae5b9460859770594bfa7c88145


,amount0,amount1,boughtToken0,price0After,price1After,blockNumber,blockPosition,blockTimestamp,txnHash,logIndex
21,50.000000,-0.039928,0,0.000799,1251.642085,16168966,-1,1.670852e+09,0xb58aee09dd66874b29889f1fb4553279e022e53fd70b...,97
20,240.011013,-0.191661,0,0.000799,1251.642803,16168966,-1,1.670852e+09,0xf7650797b933a90858f2201422cf5d3be3d90ae5b946...,159


In [281]:
swaps_.iloc[-1]

amount0                                                        50.0
amount1                                                   -0.039928
boughtToken0                                                      0
price0After                                                0.000799
price1After                                             1251.642085
blockNumber                                                16168966
blockPosition                                                    -1
blockTimestamp                                         1670852207.0
txnHash           0xb58aee09dd66874b29889f1fb4553279e022e53fd70b...
Name: 1, dtype: object

## Getting Relative Block Position
We'll need to create a table that sorts the Uniswap orders by their occurrence inside the block, since this is not provided by the graph API. Arghhh.

Block position will be findable via Alchemy's [eth_getTrasactionByHash](https://docs.alchemy.com/reference/eth-gettransactionbyhash) API method. The issue is that this will require _a lot_ of queries. And it requires a full node to support it. Arghhh.

EDIT - I was able to use `swap.logIndex` to order the transactions. Yeet!

## What next?
* Implement multiple `is_retail()` txn filters.

In [5]:

# all_pools = {}
# lowest_volume_ = 1e20 # absurdly high volume for the initial condition
# for i in range(20):
#     q = """
#     {
#         pools(
#             where: {volumeUSD_lt: %f}
#             first: 1000, orderBy: volumeUSD, orderDirection: desc
#         ) {
#             token0 {
#                 symbol
#                 id
#             }
#             token1 {
#                 symbol
#                 id
#             }
#             id
#             volumeUSD
#         }
#     }
#     """ % (lowest_volume_,)
        
#     res_ = json.loads(requests.post(
#         url,
#         json={"query": q, "variables": vars_}
#     ).content)
        
#     if ("data" not in res_) or (float(res_["data"]["pools"][-1]["volumeUSD"]) <= 0):
#         print(res_)
#         break
    
#     res_ = res_["data"]["pools"]
#     lowest_volume_ = float(res_[-1]["volumeUSD"])
#     print("lowest volume", lowest_volume_)
    
#     resnew_ = {
#         x["id"]: x
#         for x in res_
#     }
    
#     oldlen_ = len(all_pools)
#     all_pools.update(resnew_)
    
#     if oldlen_ == len(all_pools):
#         break
    
#     print(len(all_pools))

#     print("\n")
# #     pprint(res_)
    
    
# # query = '''
# # {
# #     pools(first: 2000, orderBy: volumeUSD, orderDirection: desc) {
# #         id
# #         token0 {
# #             symbol
# #         }
# #         token1 {
# #             symbol
# #         }
# #         volumeUSD
# #     }
# # }
# # '''

# # pools_ = json.loads(requests.post(
# #     url,
# #     json={"query": query}
# # ).content)["data"]["pools"]

